In [3]:
# https://www.youtube.com/watch?v=vAp6nUMrKYgfunction
function Babylonian(x; N = 10) # 10 steps 
    t = (1+x)/2
    for i = 2:N; t=(t + x/t)/2 end 
    t 
end 

Babylonian (generic function with 1 method)

In [4]:
α = π 
Babylonian(α), √α

(1.7724538509055159, 1.7724538509055159)

In [5]:
@time using Plots 
plotly() # might work next time because I am installing it now... 

 10.921173 seconds (8.44 M allocations: 585.242 MiB, 7.35% gc time, 20.63% compilation time: 70% of which was recompilation)


┌ Warning: For saving to png with the `Plotly` backend `PlotlyBase` and `PlotlyKaleido` need to be installed.
│   err = ArgumentError("Package PlotlyBase not found in current path.\n- Run `import Pkg; Pkg.add(\"PlotlyBase\")` to install the PlotlyBase package.")
└ @ Plots /home/vpoulsen/.julia/packages/Plots/gzYVM/src/backends.jl:420


Plots.PlotlyBackend()

In [7]:
# not showing, come back and fix. 
i = 0:.01:49 # start, step, end 
plot([x->Babylonian(x, N=i) for i=1:5], i, label = ["Iteration $j" for i=1:1, j=1:5])
plot!(sqrt, i, c="black", label="sqrt", title="Babylonian √")

In [9]:
# dual number again (really need to learn about this)
struct D <: Number 
    f::Tuple{Float64, Float64} # function, derivative pair 
end 

Sum rule: (x+y)' = x' + y'

Quotient rule: (x/y)' = (yx'-xy')/y^2

In [10]:
# calculus rules 
import Base: +, /, convert, promote_rule # overloading base 
+(x::D, y::D) = D(x.f .+ y.f) # sum rule (add pair of numbers involves adding function values and derivs.)
/(x::D, y::D) = D((x.f[1]/y.f[1], (y.f[1]*x.f[2] - x.f[1] * y.f[2])/y.f[1]^2)) # quotient rule
convert(::Type{D}, x::Real) = D((x, zero(x)))
promote_rule(::Type{D}, ::Type{<:Number}) = D

promote_rule (generic function with 145 methods)

In [12]:
# now applying the number pair on the babylonian (which we never rewrote).
x=49; Babylonian(D((x, 1))), (√x, .5/√x)

(D((7.0, 0.07142857142857142)), (7.0, 0.07142857142857142))

In [14]:
x=π; Babylonian(D((x, 1))), (√x, .5/√x)

(D((1.7724538509055159, 0.28209479177387814)), (1.7724538509055159, 0.28209479177387814))

In [15]:
# how does it work?

In [20]:
#using SymPy # getting an error here
#x = symbols("x")
#display("Iteraions as a function of x")
#for k = 1:5
#    display(simplify(Babylonian(x, N=k)))
#end 
#display("Derivatives as a function of x")
#for k = 1:5
#    display(simplify(diff(simplify(Babylonian(x, N=k)), x)))
#end


In [28]:
# how autodiff works 
## this is the old-school version 
function dBabylonian(x; N = 10)
    t = (1+x)/2
    tp = 1/2
    for i = 1:N;
        t = (t+x/t)/2;
        tp = (tp+(t-x*tp)/t^2)/2;
    end 
    tp
end 

dBabylonian (generic function with 1 method)

In [29]:
x=π; dBabylonian(x), .5/√x

(0.2820947917738782, 0.28209479177387814)

In [30]:
Babylonian(D((x,1)))

D((1.7724538509055159, 0.28209479177387814))

In [ ]:
# multiple dispatch 
# just given the rules for dual numbers 

In [31]:
Base.show(io::IO, x::D) = print(io, x.f[1], " + ", x.f[2], " ϵ")

In [34]:
# (a+bϵ) * (c = dϵ) = (ac) + (bc = ad)ϵ
# (a+bϵ) / (c = dϵ) = (a/c) = (bc - ad)/d^2ϵ

In [32]:
# add last two rules 
import Base: -, *
-(x::D, y::D) = D(x.f .- y.f)
*(x::D, y::D) = D((x.f[1]*y.f[1], (x.f[2]*y.f[1] + x.f[1]*y.f[2])))

* (generic function with 388 methods)

In [33]:
D((1,0))

1.0 + 0.0 ϵ

In [35]:
D((0,1))^2

0.0 + 0.0 ϵ

In [36]:
D((2,1))^2

4.0 + 4.0 ϵ

In [43]:
## what one should actually use: 
# ForwardDiff package, e.g. 
using ForwardDiff 
ForwardDiff.derivative(Babylonian, 100)

0.05